# **YOLO模型训练全流程笔记**

---




整体思路参考油管视频：https://www.youtube.com/watch?v=r0RspiLG260

注：上面视频中用的标记工具是labelstudio，但我试了一下导出效果不太稳定，所以下面换用了更为稳定的labelImg

# **1、训练环境检查**

In [ ]:
!nvidia-smi

Mon Sep  1 07:53:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **2、使用labelImg标注**

labelImg github仓库链接：https://github.com/HumanSignal/labelImg

油管教学视频链接：https://www.youtube.com/watch?v=pTJT8kKi9SM&t=53s

步骤：

1、创建虚拟环境，注意labelImg版本较老，需要匹配python3.9

2、打开terminal，输入labelImg，打开软件，开始标注

3、注意：可以先标注一部分，再加入新图片，再标注。但务必注意，不可以两个文件夹分别标注之后再合并，因为classes.txt可能会冲突！

数据集文件夹命名为data，文件夹内部结构及命名如下，方便后续调用python代码分组。

image可以不全部标注。后面会有代码用于删除冗余图片

注：发现用微信传输图片可以自动编号，非常方便！

```
data/
├── images/
│   ├── 微信图片_20250731155238_291.jpg
│   ├── 微信图片_20250731155238_292.jpg
│   ├── 微信图片_20250731155238_293.jpg
│   ├── 微信图片_20250731155238_294.jpg
│   ├── 微信图片_20250731155238_295.jpg
│   └── 微信图片_20250731155238_296.jpg
└── labels/
    ├── classes.txt
    ├── 微信图片_20250731155238_291.txt
    ├── 微信图片_20250731155238_292.txt
    ├── 微信图片_20250731155238_293.txt
    └── 微信图片_20250731155238_294.txt
```

# **3、准备训练数据集**

以data.zip的形式在本笔记中上传data文件夹。

运行以下代码，解压缩数据集data.zip，将其中的所有文件解压到 /content/custom_data 文件夹。

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data

运行以下代码，删除未标记的图片。

In [ ]:
import os
import glob

def filter_images_by_labels(images_dir, labels_dir):
    label_files = glob.glob(os.path.join(labels_dir, "*.txt"))

    label_basenames = set()
    for label_file in label_files:
        basename = os.path.splitext(os.path.basename(label_file))[0]
        label_basenames.add(basename)

    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif', '*.tiff']
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(images_dir, ext)))
        image_files.extend(glob.glob(os.path.join(images_dir, ext.upper())))

    files_to_keep = []
    files_to_delete = []

    for image_file in image_files:
        basename = os.path.splitext(os.path.basename(image_file))[0]

        if basename in label_basenames:
            files_to_keep.append(image_file)
        else:
            files_to_delete.append(image_file)

    if files_to_delete:
        for file in files_to_delete:
            try:
                os.remove(file)
            except OSError as e:
                print(f"删除失败: {os.path.basename(file)} - {e}")

    print("image与label文件名匹配完成！")

custom_data_directory = "custom_data"
data_directory = os.path.join(custom_data_directory, "data")
images_directory = os.path.join(data_directory, "images")
labels_directory = os.path.join(data_directory, "labels")

if os.path.exists(images_directory) and os.path.exists(labels_directory):
    filter_images_by_labels(images_directory, labels_directory)
else:
    print("错误：图片目录或标签目录不存在")
    print(f"图片目录存在: {os.path.exists(images_directory)}")
    print(f"标签目录存在: {os.path.exists(labels_directory)}")

image与label文件名匹配完成！


运行以下代码，将在根目录下生成一个data文件夹，内部自动按照8:1:1的比例分配train, test, val为可训练格式。

data文件夹结构如下：

```
/content/data/
├── images/
│   ├── train/
│   │   ├── 图片1.jpg
│   │   ├── 图片2.jpg
│   │   └── ... (80%的图片)
│   ├── test/
│   │   ├── 图片3.jpg
│   │   └── ... (10%的图片)
│   └── val/
│       ├── 图片4.jpg
│       └── ... (10%的图片)
└── labels/
    ├── train/
    │   ├── 图片1.txt
    │   ├── 图片2.txt
    │   └── ... (80%的标签)
    ├── test/
    │   ├── 图片3.txt
    │   └── ... (10%的标签)
    └── val/
        ├── 图片4.txt
        └── ... (10%的标签)
```

In [ ]:
import os
import shutil
import random
from pathlib import Path

source_images_dir = "/content/custom_data/data/images"
source_labels_dir = "/content/custom_data/data/labels"
output_dir = "/content/data"

if not os.path.exists(source_images_dir):
    print(f"错误：图片目录不存在 - {source_images_dir}")
    exit()

if not os.path.exists(source_labels_dir):
    print(f"错误：标签目录不存在 - {source_labels_dir}")
    exit()

image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif', '*.tiff']
image_files = []

for ext in image_extensions:
    image_files.extend(Path(source_images_dir).glob(ext))
    image_files.extend(Path(source_images_dir).glob(ext.upper()))

label_files = list(Path(source_labels_dir).glob("*.txt"))

image_label_pairs = []

for image_file in image_files:
    image_name = image_file.stem
    label_file = Path(source_labels_dir) / f"{image_name}.txt"

    if label_file.exists():
        image_label_pairs.append((image_file, label_file))

if len(image_label_pairs) == 0:
    print("没有找到匹配的图片和标签文件")
    exit()

random.seed(42)
random.shuffle(image_label_pairs)

total_count = len(image_label_pairs)
train_count = int(total_count * 0.8)
test_count = int(total_count * 0.1)
val_count = total_count - train_count - test_count

train_pairs = image_label_pairs[:train_count]
test_pairs = image_label_pairs[train_count:train_count + test_count]
val_pairs = image_label_pairs[train_count + test_count:]

# 创建COCO8格式的目录结构
output_images_dir = Path(output_dir) / "images"
output_labels_dir = Path(output_dir) / "labels"

# 创建train目录
train_images_dir = output_images_dir / "train"
train_labels_dir = output_labels_dir / "train"
train_images_dir.mkdir(parents=True, exist_ok=True)
train_labels_dir.mkdir(parents=True, exist_ok=True)

for image_file, label_file in train_pairs:
    shutil.copy2(image_file, train_images_dir / image_file.name)
    shutil.copy2(label_file, train_labels_dir / label_file.name)

# 创建test目录
test_images_dir = output_images_dir / "test"
test_labels_dir = output_labels_dir / "test"
test_images_dir.mkdir(parents=True, exist_ok=True)
test_labels_dir.mkdir(parents=True, exist_ok=True)

for image_file, label_file in test_pairs:
    shutil.copy2(image_file, test_images_dir / image_file.name)
    shutil.copy2(label_file, test_labels_dir / label_file.name)

# 创建val目录
val_images_dir = output_images_dir / "val"
val_labels_dir = output_labels_dir / "val"
val_images_dir.mkdir(parents=True, exist_ok=True)
val_labels_dir.mkdir(parents=True, exist_ok=True)

for image_file, label_file in val_pairs:
    shutil.copy2(image_file, val_images_dir / image_file.name)
    shutil.copy2(label_file, val_labels_dir / label_file.name)

print("数据集分割完成！")

数据集分割完成！


# **4、准备.yaml文件**

运行以下代码生成训练所需要的.yaml文件，同样放在根目录下。

In [ ]:
import yaml
import os

dataset_path = "/content/data"
classes_file = "/content/custom_data/data/labels/classes.txt"
output_yaml = "/content/data.yaml"

if not os.path.exists(classes_file):
    print(f"错误：classes.txt文件不存在 - {classes_file}")
    print("请确保classes.txt文件存在并包含类别名称")
    exit()

classes = []
with open(classes_file, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip()) == 0:
            continue
        classes.append(line.strip())

number_of_classes = len(classes)

config = {
    'path': dataset_path,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': number_of_classes,
    'names': classes
}

with open(output_yaml, 'w', encoding='utf-8') as f:
    yaml.dump(config, f, sort_keys=False)

print(f"YAML配置文件已创建: {output_yaml}")

print('\n文件内容:\n')
print(open(output_yaml).read())

YAML配置文件已创建: /content/data.yaml

文件内容:

path: /content/data
train: images/train
val: images/val
test: images/test
nc: 2
names:
- drink
- food



到此已经准备好完整的训练数据集。

```
/content/
├── data/
│   ├── images/
│   │   ├── train/
│   │   │   ├── 微信图片_20250731155238_291.jpg
│   │   │   ├── 微信图片_20250731155238_292.jpg
│   │   │   ├── 微信图片_20250731155238_293.jpg
│   │   │   └── 微信图片_20250731155238_294.jpg
│   │   ├── test/
│   │   │   └── 微信图片_20250731155238_295.jpg
│   │   └── val/
│   │       └── 微信图片_20250731155238_296.jpg
│   └── labels/
│       ├── train/
│       │   ├── 微信图片_20250731155238_291.txt
│       │   ├── 微信图片_20250731155238_292.txt
│       │   ├── 微信图片_20250731155238_293.txt
│       │   └── 微信图片_20250731155238_294.txt
│       ├── test/
│       │   └── 微信图片_20250731155238_295.txt
│       └── val/
│           └── 微信图片_20250731155238_296.txt
└── data.yaml
```

# **5、准备训练环境**

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.6 MB/s eta 0:00:00


# **6、开始训练**

运行以下代码开始训练。其中代码意义如下，可自定义修改。

```
model=yolov8n.pt：指定预训练模型
epochs=10：训练轮数
imgsz=640：输入图片尺寸
```

In [ ]:
!yolo detect train data=/content/data.yaml model=yolov8n.pt epochs=10 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.190 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_widt

默认情况下，YOLOv8会将训练结果保存在/content/runs/detect/train/。该目录包含：

```
模型文件：best.pt（最佳模型）、last.pt（最后一轮模型）
训练日志：results.csv、results.png
验证结果：验证图片和预测结果
配置文件：训练配置和超参数
```

# **7、下载数据集和训练成果**

**（1）下载完整项目包**


---



ultralyticsHUB平台提供了在线demo服务，因此下载数据集data后，可以自行压缩上传到HUB的data平台上训练，方便直接查看效果。

另外自己测试了一下，用ultralyticsHUB平台直接训练后下载模型会比这里下载的效果更好，可能是平台自定义的参数更大导致的，所以也可以用这个笔记获得数据集之后在平台上训练

平台使用我后面会出一个视频，自己探索一下是有下载.pt文件的功能的

In [ ]:
# 创建完整项目包
!mkdir /content/myproject 2>/dev/null
!cp -r /content/data /content/myproject/ 2>/dev/null
!cp /content/data.yaml /content/myproject/data 2>/dev/null
!cp /content/runs/detect/train/weights/best.pt /content/myproject/my_model.pt 2>/dev/null

# 将myproject文件夹压缩
!cd myproject && zip -qq -r /content/my_project.zip .

下面这一步利用代码下载，可能花的时间比较长，也可以直接从边栏点击···下载。

In [ ]:
from google.colab import files
files.download('/content/my_project.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**（2）仅下载数据集（用于上传ultralyticsHUB训练模型）**

---

[ultralyticsHUB指路](https://hub.ultralytics.com/)

在data文件夹中写符合ultralyticsHUB数据集要求的.yaml文件

HUB中有dataset的示例，其yaml文件为了适配HUB自己的代码写法和我们这里的不太一样，所以要另外写一个保存！

In [ ]:
import yaml
import os

dataset_path = "/content/data"
classes_file = "/content/custom_data/data/labels/classes.txt"
output_yaml = "/content/data/dataset.yaml"

if not os.path.exists(classes_file):
    print(f"错误：classes.txt文件不存在 - {classes_file}")
    print("请确保classes.txt文件存在并包含类别名称")
    exit()

classes = []
with open(classes_file, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip()) == 0:
            continue
        classes.append(line.strip())

number_of_classes = len(classes)

config = {
    'path': '',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': number_of_classes,
    'names': classes
}

with open(output_yaml, 'w', encoding='utf-8') as f:
    yaml.dump(config, f, sort_keys=False)

print(f"YAML配置文件已创建: {output_yaml}")

print('\n文件内容:\n')
print(open(output_yaml).read())

YAML配置文件已创建: /content/data/dataset.yaml

文件内容:

path: ''
train: images/train
val: images/val
test: images/test
nc: 2
names:
- drink
- food



压缩data文件夹至dataset.zip

In [ ]:
!cd data && zip -qq -r /content/dataset.zip .

下载dataset.zip文件

In [ ]:
from google.colab import files
files.download('/content/dataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **8、在本地调用模型**

本地调用模型需要预先安装pytorch和ultralytics两个库。

尽量使用py3.11+的虚拟环境。注意该虚拟环境和labelImg的不能是同一个，因为labelImg只能匹配py3.9解释器。

从官网获取下载pytorch的适配命令行：https://pytorch.org/

ultralytics安装命令行：pip3 install ultralytics

另外conda命令似乎无法下载这两个库，最好使用pip。

调用示例链接：https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/examples/candy_calorie_counter/candy_calorie_counter.py

# **备用**

从边栏删除文件夹的命令行代码

In [ ]:
!rm -rf /content/data